In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

!pip install apyori
from apyori import apriori

In [ ]:
!pip install pyspark
import pyspark
from pyspark import SparkContext

# Spark Context 
# Also extra coniguration can be added 
sc = SparkContext.getOrCreate()

     |████████████████████████████████| 204.8MB 80kB/s 
     |████████████████████████████████| 204kB 22.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=3cc991512e6b0bde71f1785eed891588dd111c5f039d0169a49efe9e0c3eeeb8
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [ ]:
file =  sc.textFile("/content/store_data.csv")

print(file.collect())

['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil', 'burgers,meatballs,eggs,,,,,,,,,,,,,,,,,', 'chutney,,,,,,,,,,,,,,,,,,,', 'turkey,avocado,,,,,,,,,,,,,,,,,,', 'mineral water,milk,energy bar,whole wheat rice,green tea,,,,,,,,,,,,,,,', 'low fat yogurt,,,,,,,,,,,,,,,,,,,', 'whole wheat pasta,french fries,,,,,,,,,,,,,,,,,,', 'soup,light cream,shallot,,,,,,,,,,,,,,,,,', 'frozen vegetables,spaghetti,green tea,,,,,,,,,,,,,,,,,', 'french fries,,,,,,,,,,,,,,,,,,,', 'eggs,pet food,,,,,,,,,,,,,,,,,,', 'cookies,,,,,,,,,,,,,,,,,,,', 'turkey,burgers,mineral water,eggs,cooking oil,,,,,,,,,,,,,,,', 'spaghetti,champagne,cookies,,,,,,,,,,,,,,,,,', 'mineral water,salmon,,,,,,,,,,,,,,,,,,', 'mineral water,,,,,,,,,,,,,,,,,,,', 'shrimp,chocolate,chicken,honey,oil,cooking oil,low fat yogurt,,,,,,,,,,,,,', 'turkey,eggs,,,,,,,,,,,,,,,,

In [ ]:
## Splited items  
lblitems = file.map(lambda line: line.split(','))

print(lblitems.collect())


[['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil'], ['burgers', 'meatballs', 'eggs', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['chutney', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['turkey', 'avocado', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['low fat yogurt', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['whole wheat pasta', 'french fries', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['soup', 'light cream', 'shallot', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
## Whole lines in single array 
items = file.flatMap(lambda line:line.split(','))

print(items.collect())

['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil', 'burgers', 'meatballs', 'eggs', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'chutney', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'turkey', 'avocado', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'low fat yogurt', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'whole wheat pasta', 'french fries', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'soup', 'light cream', 'shallot', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [ ]:
## Unique frequent items in dataset
uniqueItems = items.distinct()

# Add 1 as Tuple
supportRdd = items.map(lambda item: (item , 1))

# Method for sum in reduceByKey method
def sumOparator(x,y):
    return x+y

# Sum of values by key
supportRdd = supportRdd.reduceByKey(sumOparator)

print(supportRdd.collect())


# First support values
supports = supportRdd.map(lambda item: item[1]) # Return only support values


[('shrimp', 10), ('green grapes', 2), ('whole weat flour', 1), ('cottage cheese', 3), ('green tea', 17), ('honey', 5), ('salad', 2), ('antioxydant juice', 2), ('frozen smoothie', 8), ('olive oil', 8), ('burgers', 16), ('', 2272), ('turkey', 13), ('milk', 13), ('whole wheat rice', 3), ('light cream', 3), ('shallot', 3), ('spaghetti', 30), ('cookies', 11), ('cooking oil', 9), ('chocolate', 24), ('extra dark chocolate', 2), ('red wine', 3), ('pasta', 9), ('sparkling water', 3), ('ham', 4), ('toothpaste', 1), ('parmesan cheese', 1), ('herb & pepper', 7), ('magazines', 2), ('strawberries', 2), ('strong cheese', 1), ('pickles', 2), ('brownies', 2), ('cereals', 1), ('light mayo', 2), ('soda', 1), ('cider', 2), ('yogurt cake', 1), ('butter', 1), ('french wine', 2), ('salt', 1), ('barbecue sauce', 1), ('mayonnaise', 1), ('mushroom cream sauce', 1), ('almonds', 6), ('avocado', 9), ('vegetables mix', 1), ('yams', 2), ('energy drink', 5), ('tomato juice', 6), ('low fat yogurt', 11), ('mineral wate

In [ ]:
# Define minimum support value 
minSupport = supports.min()

# If mininmum support is 1 then replace it with 2 
minSupport = 2 if minSupport == 1 else minSupport

## Filter first supportRdd with minimum support 
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

## Craete base RDD with will be updated every iteration
baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
print('1 . Table has crated...') 

supportRdd = supportRdd.map(lambda item: item[0])
supportRddCart = supportRdd

1 . Table has crated...


In [ ]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [ ]:
c = 2 # Combination length 

while(supportRdd.isEmpty() == False):

    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(lblitems)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(sumOparator)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
    c = c+1 

In [ ]:
class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    # Check Items sets includes at least one comman item // Example command: # any(l == k for k in z for l in x )
    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        # Parent item list
        parent = set(item[0][0])
        
        # Child item list
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        # Parent and Child support values
        parentSupport = item[0][1]
        childSupport = item[1][1]
        # Finds the item set confidence is going to be found

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

        
# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

# Create Filter Object
ff = Filter()

#deneme = calcuItems.map(lambda item: lens(item)) 
total = calcuItems.count()

print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

  
print(baseRddConfidence.collect())

Py4JJavaError: ignored